In [43]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../") 
import models.utility as utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [309]:
import pandas as pd
from sklearn import preprocessing as preprocessing
import sklearn.linear_model as linear_model
import sklearn.neighbors as neighbors
import sklearn.svm as svm
import sklearn.model_selection as model_selection
import sklearn.metrics as metrics
from sklearn.metrics import brier_score_loss
from sklearn.calibration import CalibratedClassifierCV

ModuleNotFoundError: No module named 'mpl_toolkits.basemap'

In [178]:
def get_dummies(frame, col):
    one_hot_encoded = pd.get_dummies(frame[col])
    one_hot_encoded = pd.concat((frame, one_hot_encoded), axis=1)
    return one_hot_encoded.drop(col, axis=1)

In [2]:
raw_articles, raw_annotations, raw_locations = utils.load_dataset()

articles count: 9959
annotations count: 3221
locations count: 3562


In [76]:
raw_articles[0]["annotation_uri"]

['13b906e9b805ed29d8dc62e7d20d0705']

In [3]:
uri_index = utils.get_annotation_uri_index(raw_annotations, raw_locations)
location_index = utils.get_location_index(raw_locations)
article_to_loc_index = utils.get_article_to_loc_index(raw_articles, raw_locations, uri_index)

In [75]:

uri_index['be48009bf7ec91d46983f1044db47f76']['locations']

[{'iid': '7fa1c86a-591d-46ea-a1f7-fec79e667b22',
  'council_annotation_uri': '0280893123d7785441eff9fa1641170b',
  'county': 'Derbyshire',
  'local_government_area': 'City of Derby',
  'nuts_region': 'East Midlands',
  'lng': -1.44377,
  'lat': 52.91562,
  'country': 'England',
  'name': 'Pride Park',
  'lastModifiedTime': 1516791066877,
  'origin_type': 'Other Settlement',
  'annotation_uri': ['be48009bf7ec91d46983f1044db47f76'],
  'id': 'ec79cba6-d6a6-485d-ab33-65ac0412f54b',
  'type': 'locations',
  'postcode_sector': 'DE24 8'}]

In [173]:
locations = pd.DataFrame(raw_locations)[["id", "lat", "lng", "nuts_region"]]
locations.set_index("id", inplace=True)

In [174]:
locations.head()

,lat,lng,nuts_region
id,,,
ec79cba6-d6a6-485d-ab33-65ac0412f54b,52.91562,-1.44377,East Midlands
64be7f94-cfcc-49c9-88f0-0dcace732a0f,51.44012,-2.61655,South West
5ceeb5cc-e1e4-4011-a3d8-3f874b793915,53.72422,-2.48623,North West
eeb33821-a955-403b-b732-5f5c169370eb,52.46249,-1.85391,West Midlands
19ee6c1b-75bb-4c85-bd39-22367e9fb9d3,53.65412,-1.75504,Yorkshire and the Humber


In [175]:
locations = get_dummies(locations, "nuts_region")

In [191]:
pub_feature_set = []
#columns = ["publisher", "lat", "lng", "origin_type", "name", "nuts_region", "local_government_area", "id"]
publishers = []
for article in raw_articles:
    for annotation in article["annotation_uri"]:
        if annotation in uri_index.keys():
            locs = uri_index[annotation]["locations"]
            for loc in locs:
                publishers.append(utils.clean_publisher(article["publisher"]))
                pub_feature_set.append(locations.loc[loc["id"], :])
pub_feature_set = pd.DataFrame(pub_feature_set)
pub_feature_set["publisher"] = publishers

In [193]:
pub_feature_set.head(2)

,lat,lng,East Midlands,Eastern,Isle of Man,London,North East,North West,Northern Ireland,Scotland,South East,South West,Wales,West Midlands,Yorkshire and the Humber,publisher
bd5a1b69-a99a-4934-9441-98fad30ec993,51.51437,-0.09229,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nwemail
eeb33821-a955-403b-b732-5f5c169370eb,52.46249,-1.85391,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,rotherhamadvertiser


In [179]:

#dropped_cols  = ['country', 'county', 'iid', 'local_government_area', 'name', 'origin_type', 'postcode_sector']
#pub_feature_set.drop(dropped_cols, inplace=True, axis=1)

In [180]:
n = 100
publishers = clean_features_set["publisher"].unique()
publishers.sort()
large_publishers = publishers[pub_feature_set.groupby("publisher")["publisher"].count() > n]
print(large_publishers)

['belfastlive' 'birminghammail' 'blackpoolgazette' 'bournemouthecho'
 'chroniclelive' 'coventrytelegraph' 'dailyrecord' 'devonlive'
 'eastangliandailytimes' 'easterndailypress' 'gazettelive' 'getsurrey'
 'glasgowlive' 'inyourarea' 'lep' 'liverpoolecho' 'manchestereveningnews'
 'mirror' 'plymouthherald' 'somersetlive' 'theargus' 'thenorthernecho'
 'thestar' 'thisislocallondon' 'walesonline' 'yorkshirepost']


In [181]:
clean_features_set.columns

Index(['lat', 'lng', 'publisher', 'East Midlands', 'Eastern', 'Isle of Man',
       'London', 'North East', 'North West', 'Northern Ireland', 'Scotland',
       'South East', 'South West', 'Wales', 'West Midlands',
       'Yorkshire and the Humber'],
      dtype='object')

In [182]:
pub_feature_set.head(2)

,lat,lng,East Midlands,Eastern,Isle of Man,London,North East,North West,Northern Ireland,Scotland,South East,South West,Wales,West Midlands,Yorkshire and the Humber,publisher
0,51.51437,-0.09229,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,nwemail
1,52.46249,-1.85391,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,rotherhamadvertiser


In [276]:
dataset = pub_feature_set[pub_feature_set["publisher"] == 'birminghammail']
def form_sample(dataset, locations):
    known_ids = set(dataset.index.values)
    dataset.drop("publisher", axis=1, inplace=True)
    dataset.reset_index(inplace=True, drop=True)

    selected_ids = utils.sample_locations(raw_locations, dataset.shape[0], known_ids)
    
    negative_sample = locations.loc[selected_ids].reset_index(drop=True).copy()
    negative_sample["Y"] = 0
    positive_sample = dataset.copy()
    positive_sample["Y"] = 1
    sample = pd.concat((positive_sample, negative_sample))
    return sample

In [206]:
sample.head()

,lat,lng,East Midlands,Eastern,Isle of Man,London,North East,North West,Northern Ireland,Scotland,South East,South West,Wales,West Midlands,Yorkshire and the Humber,Y
0,51.56452,-0.10892,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,52.43788,-1.78288,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
2,52.41275,-1.77755,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
3,52.47928,-1.90294,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
4,51.51279,-0.20570,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [239]:
# def evaluate(features, clf):
#     known_ids = set(features["id"].values)
#     selected_ids = utils.sample_locations(raw_locations, features.shape[0], known_ids)
#     negative_sample = locations[selected_ids]
#     negative_sample["Y"] = 0
#     positive_sample = features[cols].copy()
#     positive_sample["Y"] = 1
#     sample = pd.concat((positive_sample, negative_sample))
#     X = sample.drop("Y", axis=1)
#     Y = sample["Y"]
#     train, test = model_selection.train_test_split(sample.drop("id", axis=1), test_size=0.3)
#     model = clf.fit(train.drop("Y"), y=train["Y"])
#     y_hat = model.predict(test.drop("Y"))
#     print(pd.DataFrame(metrics.confusion_matrix(test["Y"], y_hat), index=["true neg", "true pos"], columns=["pred pos", "pred neg"]))
#     print(metrics.classification_report(test["Y"], y_hat))

In [221]:
publisher = "mirror"

In [313]:
for publisher in ["birminghammail"]: #large_publishers:
    print("\n-------------------------------------------------")
    print(f"Publisher: {publisher}\n")
    dataset = pub_feature_set[pub_feature_set["publisher"] == publisher]
    sample = form_sample(dataset, locations)
    clf = neighbors.KNeighborsClassifier()
    clf_sigmoid = CalibratedClassifierCV(clf, cv=2, method='sigmoid')
    train, test = model_selection.train_test_split(sample, test_size=0.3)
    
    model = clf_sigmoid.fit(train.drop("Y", axis=1), y=train["Y"])
    y_hat = model.predict(test.drop("Y", axis=1))
    y_hat_prob = model.predict_proba(test.drop("Y", axis=1))
    print(pd.DataFrame(metrics.confusion_matrix(test["Y"], y_hat), 
                       index=["true neg", "true pos"], 
                       columns=["pred neg", "pred pos"]))
    print(metrics.classification_report(test["Y"], y_hat))


-------------------------------------------------
Publisher: birminghammail

          pred neg  pred pos
true neg       116         5
true pos        15       110
             precision    recall  f1-score   support

          0       0.89      0.96      0.92       121
          1       0.96      0.88      0.92       125

avg / total       0.92      0.92      0.92       246



/home/vagrant/.local/share/virtualenvs/vagrant-gKDsaKU3/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [314]:
y_hat

array([1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0,
       1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 1, 0])

In [315]:
y_hat_prob

array([[0.03874982, 0.96125018],
       [0.96420828, 0.03579172],
       [0.91299712, 0.08700288],
       [0.25763682, 0.74236318],
       [0.03874982, 0.96125018],
       [0.03874982, 0.96125018],
       [0.0923724 , 0.9076276 ],
       [0.91299712, 0.08700288],
       [0.96420828, 0.03579172],
       [0.03874982, 0.96125018],
       [0.03874982, 0.96125018],
       [0.93151824, 0.06848176],
       [0.93151824, 0.06848176],
       [0.96420828, 0.03579172],
       [0.96420828, 0.03579172],
       [0.93151824, 0.06848176],
       [0.03874982, 0.96125018],
       [0.96420828, 0.03579172],
       [0.96420828, 0.03579172],
       [0.03874982, 0.96125018],
       [0.03874982, 0.96125018],
       [0.03874982, 0.96125018],
       [0.96420828, 0.03579172],
       [0.83877177, 0.16122823],
       [0.93151824, 0.06848176],
       [0.83877177, 0.16122823],
       [0.03874982, 0.96125018],
       [0.93151824, 0.06848176],
       [0.91299712, 0.08700288],
       [0.93151824, 0.06848176],
       [0.

In [292]:
(sample["Y"] == 0).sum()

199

In [267]:
(test["Y"] == 1).sum()

199

In [269]:
sample.shape[0]

519